In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import json

from PIL import Image
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
from datetime import datetime
import os
from torch.utils import data
import random
import copy
import itertools
import io
import uuid
from sklearn.model_selection import KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

import wandb
wandb_username = 'denizjafari'
local_username = 'denizjafari'


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0') 
else:
    device = torch.device('cpu')
print(device)

cuda:0


In [3]:
# source: https://scikit-learn.org/0.16/auto_examples/model_selection/plot_confusion_matrix.html
#example-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm,fold, classnames, title='Confusion matrix', cmap=plt.cm.Blues):
    fig, ax = plt.subplots(1, figsize=(14, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classnames))
    plt.xticks(tick_marks, classnames, rotation=45)
    plt.yticks(tick_marks, classnames)
    
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=19)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('ViewNetConfMtrx' + fold + '.png')
    wandb.log({"confusion"+ fold: plt})

In [4]:
# root directory
root_dir = "/home/andreasabo/Documents/HNProject/"
split_file_base = "/home/andreasabo/Documents/HNUltra/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/" + local_username + "/Documents/HNProject/all_label_img/"

# read target df
csv_path = os.path.join(root_dir, "all_splits_1000000.csv")
data_df = pd.read_csv(csv_path, usecols=['subj_id', 'image_ids', 'view_label', 'view_train'])

# Are we doing the final test?
test_data = False

### **Reading Data Indicies and Labels**

In [5]:
label_mapping = {'Other':0, 'Saggital_Right':1, 'Transverse_Right':2, 
                 'Saggital_Left':3, 'Transverse_Left':4, 'Bladder':5}
label_unmapping = {0: 'Other', 1:'Saggital_Right', 2: 'Transverse_Right', 
                   3:'Saggital_Left', 4:'Transverse_Left', 5: 'Bladder'}

data_df['view_label'] = data_df['view_label'].map(label_mapping)

train_df = data_df[data_df.view_train == 1]
test_df = data_df[data_df.view_train == 0]

unique_subj = train_df.subj_id.unique()

# Create the splits for 5-fold cross validation based on subj_id
data_split_file = split_file_base + 'data_splits.json'
if not os.path.isfile(data_split_file):

    kf = KFold(n_splits=5, random_state=0, shuffle=True)
    fold = 0
    all_folds = {}
    for train_subj, val_subj in kf.split(unique_subj):
        train_ids  = unique_subj[train_subj]
        val_ids = unique_subj[val_subj]

        train_images = train_df[train_df.subj_id.isin(train_ids)].image_ids.tolist()
        val_images = train_df[train_df.subj_id.isin(val_ids)].image_ids.tolist()
        train_labels = train_df[train_df.subj_id.isin(train_ids)].view_label.tolist()
        val_labels = train_df[train_df.subj_id.isin(val_ids)].view_label.tolist()
        cur_fold = {'train_ids': train_images, 'valid_ids': val_images, 'train_labels': train_labels, 'valid_labels': val_labels}
        all_folds[fold] = cur_fold
        fold += 1

    print("Saving data splits")
    with open(data_split_file, 'w') as f:
        json.dump(all_folds, f)
        
else: # just load from file
    print("Reading splits from file")
    with open(data_split_file, 'r') as f:
        all_folds = json.load(f)

# If we're testing, overwrite the training data with the entire train/test data
#if test_data:
#    train_images = train_df.image_ids.tolist()
test_images = test_df.image_ids.tolist()
#    train_labels = train_df.view_label.tolist()
test_labels = test_df.view_label.tolist()

test_set = {'test_ids': test_images, 'test_labels': test_labels}
    
#all_folds['test'] = cur_fold

Reading splits from file


In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [7]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    elif model_name == 'viewnet':
        conv1_filters = 8
        conv2_filters = 16
        conv3_filters = 32
        linear1_size = 512
        dropout = 0.25
        model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
        input_size = 256
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


class ViewNet(nn.Module):

    def __init__(self, num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout):
        super(ViewNet, self).__init__()
        self.conv1_filters = conv1_filters
        self.conv2_filters = conv2_filters
        self.conv3_filters = conv3_filters
        self.linear1_size = linear1_size
        self.drop_percent = dropout
        self.max_pool = 4
        self.conv_output = int(self.conv3_filters*(256/(self.max_pool**3))*(256/(self.max_pool**3)))
        print("conv_output: ", self.conv_output)

        
        self.conv1 = nn.Conv2d(1, self.conv1_filters, 4, padding=2)
        self.conv2 = nn.Conv2d(self.conv1_filters, self.conv2_filters, 4, padding=2)
        self.conv3 = nn.Conv2d(self.conv2_filters, self.conv3_filters, 4, padding=2)
        self.pool = nn.MaxPool2d(self.max_pool, self.max_pool)
        self.dropout = nn.Dropout(self.drop_percent)
        self.linear1 = nn.Linear(self.conv_output, self.linear1_size)
        self.linear2 = nn.Linear(self.linear1_size, num_classes)
    
    def forward(self, x):
        x = self.pool(self.dropout(F.relu(self.conv1(x))))
        x = self.pool(self.dropout(F.relu(self.conv2(x))))
        x = self.pool(self.dropout(F.relu(self.conv3(x))))
        x = x.view(-1, self.conv_output) 
        x = self.dropout(F.relu((self.linear1(x))))
        x = self.linear2(x)
        return x

In [8]:
# Code from: https://gist.github.com/stefanonardo/693d96ceb2f531fa05db530f3e21517d
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=True):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [9]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False, final_testing=False):
    es = EarlyStopping(patience = 15)
    stop_now = 0

    since = time.time()
    classnames = ['Other', 'Saggital_Right', 'Transverse_Right', 'Saggital_Left','Transverse_Left', 'Bladder']
    val_acc_history = []
    
    val_metrics_list = []
    train_metrics_list = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epoch_with_best_val_acc = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 54)

        if stop_now:
            break
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            running_preds = []
            running_labels = []

            # Iterate over data.
            for inputs, labels, _ in dataloaders[phase]:
                labels = labels.type(torch.long)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        labels = torch.argmax(labels, 1)
                        running_preds += torch.argmax(outputs, 1).tolist()
                        running_labels += labels.tolist()
                        loss = criterion(outputs, labels)

                    preds = torch.argmax(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} loss:\t{:.4f} | {} acc:\t{:.4f}\n'.format(phase, epoch_loss, phase, epoch_acc))

            if phase == 'train':
                wandb.log({'epoch': epoch, 'train_acc':epoch_acc, 'train_loss':epoch_loss})
                
                cur_train_metrics = {}
                                # compute and log f1, precision, recall for each class
                for c in range(6):
                    running_labels = np.asarray(running_labels)
                    running_preds = np.asarray(running_preds)

                    cur_c_labs_bin = np.asarray([0] *len(running_labels))
                    cur_c_preds_bin = np.asarray([0] *len(running_labels))

                    # Need to binarize
                    cur_c_preds_bin[running_preds == c] = 1
                    cur_c_labs_bin[running_labels == c] = 1
                    f1 = f1_score(cur_c_labs_bin, cur_c_preds_bin)
                    precision = precision_score(cur_c_labs_bin, cur_c_preds_bin)
                    recall = recall_score(cur_c_labs_bin, cur_c_preds_bin)
                    
                    cur_train_metrics['train_' + label_unmapping[c] + '_f1'] = f1
                    cur_train_metrics['train_' + label_unmapping[c] + '_precision'] = precision
                    cur_train_metrics['train_' + label_unmapping[c] + '_recall'] = recall
                    
                
                train_metrics_list.append(cur_train_metrics)
                
                average_types = ['macro', 'micro', 'weighted']
                average_metrics_to_log = ['precision', 'recall', 'f1score', 'support']
                average_dict = {'epoch': epoch}
                for av in average_types:
                    results_tuple = precision_recall_fscore_support(running_labels, running_preds, average=av)
                    for m in range(len(average_metrics_to_log)):      
                        average_dict[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                        cur_train_metrics[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                cur_train_metrics[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)                  
                average_dict[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)     
                wandb.log(cur_train_metrics)
                
            if phase == 'val':
                wandb.log({'valid_loss':epoch_loss, 'valid_acc':epoch_acc, 'epoch': epoch})
               
            
                cur_val_metrics = {}
                # compute and log f1, precision, recall for each class
                for c in range(6):
                    running_labels = np.asarray(running_labels)
                    running_preds = np.asarray(running_preds)

                    cur_c_labs_bin = np.asarray([0] *len(running_labels))
                    cur_c_preds_bin = np.asarray([0] *len(running_labels))

                    # Need to binarize
                    cur_c_preds_bin[running_preds == c] = 1
                    cur_c_labs_bin[running_labels == c] = 1
                    f1 = f1_score(cur_c_labs_bin, cur_c_preds_bin)
                    precision = precision_score(cur_c_labs_bin, cur_c_preds_bin)
                    recall = recall_score(cur_c_labs_bin, cur_c_preds_bin)
                    wandb.log({'valid_' + label_unmapping[c] + '_f1': f1})
                    wandb.log({'valid_' + label_unmapping[c] + '_precision': precision})
                    wandb.log({'valid_' + label_unmapping[c] + '_recall': recall})
                
                    cur_val_metrics['val_' + label_unmapping[c] + '_f1'] = f1
                    cur_val_metrics['val_' + label_unmapping[c] + '_precision'] = precision
                    cur_val_metrics['val_' + label_unmapping[c] + '_recall'] = recall
                
                average_types = ['macro', 'micro', 'weighted']
                average_metrics_to_log = ['precision', 'recall', 'f1score']
                average_dict = {'epoch': epoch}
                for av in average_types:
                    results_tuple = precision_recall_fscore_support(running_labels, running_preds, average=av)
                    for m in range(len(average_metrics_to_log)):      
                        average_dict[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                        cur_val_metrics[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                cur_val_metrics[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)                  
                average_dict[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)     
                print(cur_val_metrics)
                wandb.log(cur_val_metrics)
                
                
                val_metrics_list.append(cur_val_metrics)
                
            if phase == 'train':
                print(classification_report(running_labels, running_preds))
                train_acc = epoch_acc
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_acc_train = train_acc
                epoch_with_best_val_acc = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), os.path.join(wandb.run.dir, "model.pt"))
                print(classification_report(running_labels, running_preds))

            if phase == 'val':
                val_acc_history.append(epoch_acc)
                if es.step(epoch_loss) and not final_testing:
                    stop_now = 1
                    print("EARLY STOPPING " + str(epoch))
                    break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}\n'.format(best_acc))
    
    # Directly save the best results in this fold
    wandb.config.best_acc = best_acc
    wandb.config.best_epoch = epoch_with_best_val_acc

    wandb.config.val_acc_history = val_acc_history
    wandb.config.best_epoch = epoch_with_best_val_acc
    
    wandb.config.update(val_metrics_list[epoch_with_best_val_acc])
    wandb.config.update(train_metrics_list[epoch_with_best_val_acc])
    
    metrics_from_best_epoch = {'best_epoch': epoch_with_best_val_acc, 'last_epoch': epoch}
    metrics_from_best_epoch.update( val_metrics_list[epoch_with_best_val_acc] )
    metrics_from_best_epoch.update( train_metrics_list[epoch_with_best_val_acc] )
    metrics_from_best_epoch.update( {'val_acc': best_acc.data.cpu(), 'train_acc': best_acc_train.data.cpu()} )    
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, metrics_from_best_epoch

In [10]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels, transformations=None):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.transformations = transformations
        
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        img_path = data_dir + ID + '.jpg'
        image = Image.open(img_path).convert('L')
        
        if self.transformations:
            image = self.transformations(image)
        
        image = ToTensor()(image)
        
        y = torch.FloatTensor([0]*6)        
        y[int(self.labels[index])] = 1

        return image, y, ID

In [11]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception, viewnet]
model_name = "viewnet"

# Number of classes in the dataset: right_sag, right_trav, left_sag, left_trav, bladder, other
num_classes = 6

# Batch size for training (change depending on how much memory you have)
batch_size = 100

# Number of epochs to train for
num_epochs = 100

# Flag for feature extracting. When False, we finetune the whole model; when True we only update the reshaped layer params
feature_extract = False

# Flag for whether or not to use pretrained model
pretrain = False

In [12]:
def train5fold(network_configs, model_ft, lr, wd, amsgrad, i):
    #folds = ['test']
    folds = ['0', '1', '2', '3', '4']
    classnames = ['Other', 'Saggital_Right', 'Transverse_Right', 'Saggital_Left','Transverse_Left', 'Bladder']
    project_name = 'hnultra_test_apr6_vae'
    random_str = str(uuid.uuid4()).split("-")[0]
    best_metrics_per_fold = []
    test_metrics_per_fold = []
    model_base = copy.deepcopy(model_ft)
    for fold in folds:

        now = datetime.now()
        date_time = now.strftime("%d-%m-%Y.%H:%M:%S")
        wandb.init(project=project_name, entity=wandb_username, name=local_username + '_fold_' + fold, group=random_str)
        partition = all_folds[fold]

        model_ft = copy.deepcopy(model_base)
        model_ft = model_ft.to(device)
        wandb.watch(model_ft)

        # Gather the parameters to be optimized/updated in this run. If we are
        #  finetuning we will be updating all parameters. However, if we are
        #  doing feature extract method, we will only update the parameters
        #  that we have just initialized, i.e. the parameters with requires_grad
        #  is True.
        params_to_update = model_ft.parameters()
        #print("Params to learn:")
        if feature_extract:
            params_to_update = []
            for name,param in model_ft.named_parameters():
                if param.requires_grad == True:
                    params_to_update.append(param)
                    print("\t",name)
        else:
            for name,param in model_ft.named_parameters():
                if param.requires_grad == True:
                    print("\t",name)

        # Observe that all parameters are being optimized
        optimizer_ft = optim.Adam(params_to_update, lr=lr, weight_decay=wd, amsgrad=amsgrad)

        # Setup the loss fxn
        criterion = nn.CrossEntropyLoss()

        shuffle = True
        num_workers = 0
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}

        config_dict = {'i': i, 'batch_size': batch_size, 'shuffle': shuffle, 'num_workers': num_workers, 'fold': fold,
                       'lr': lr, 'wd': wd, 'amsgrad': amsgrad, 'model_name': model_name, 'num_classes': num_classes, 
                       'num_epochs': num_epochs, 'feature_extract': feature_extract, "pretrain": pretrain }

        wandb.config.update(config_dict)
        wandb.config.update(network_configs)
        # Tranforms
        trans = transforms.Compose([transforms.RandomAffine(degrees=8, translate=(0.1, 0.1), scale=(0.95,1.25))])

        # Generators
        training_set = Dataset(partition['train_ids'], partition['train_labels'], transformations=trans)
        training_generator = data.DataLoader(training_set, **params)

        validation_set = Dataset(partition['valid_ids'], partition['valid_labels'])
        validation_generator = data.DataLoader(validation_set, **params)
        
        testing_set = Dataset(test_set['test_ids'], test_set['test_labels'])
        test_generator = data.DataLoader(testing_set, **params)

        dataloaders_dict = {'train':training_generator, 'val':validation_generator}

        # Train & Evaluate
        model_ft, hist, metrics_from_best_epoch = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs, is_inception=(model_name=="inception"))
        best_metrics_per_fold.append(metrics_from_best_epoch)
        
        # Test the model on test data
        # saving the results in df for further analysis on machine and year
        final_df = pd.DataFrame()
        run_y = []
        run_pred = []
        run_ids = []
        running_loss =  0
        running_corrects = 0
        
        model_ft.eval()
        for inputs, labels, ids in test_generator:
            
            run_ids += np.array(ids).tolist()
            
            labels = labels.type(torch.long)
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            # zero the parameter gradients
            optimizer_ft.zero_grad()
            outputs = model_ft(inputs)
            labels = torch.argmax(labels, 1)
            run_pred += torch.argmax(outputs, 1).tolist()
            run_y += labels.tolist()
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        test_loss = running_loss / len(test_generator)
        test_acc = running_corrects.double() / len(test_generator)
        
        
        print('Test loss:\t{:.4f} | Test acc:\t{:.4f}\n'.format( test_loss, test_acc))
        wandb.log({'test_loss':test_loss, 'test_acc':test_acc})
        
        # calculating the metrics
        test_metrics = {'fold': fold}
        # compute and log f1, precision, recall for each class
        for c in range(6):
            run_ids = np.asarray(run_ids)
            run_y = np.asarray(run_y)
            run_pred = np.asarray(run_pred)
            conf_mx = confusion_matrix(run_y, run_pred)
            cur_c_labs_bin = np.asarray([0] *len(run_y))
            cur_c_preds_bin = np.asarray([0] *len(run_y))

            # Need to binarize
            cur_c_preds_bin[run_pred == c] = 1
            cur_c_labs_bin[run_y == c] = 1
            f1 = f1_score(cur_c_labs_bin, cur_c_preds_bin)
            precision = precision_score(cur_c_labs_bin, cur_c_preds_bin)
            recall = recall_score(cur_c_labs_bin, cur_c_preds_bin)

            test_metrics['test_' + label_unmapping[c] + '_f1'] = f1
            test_metrics['test_' + label_unmapping[c] + '_precision'] = precision
            test_metrics['test_' + label_unmapping[c] + '_recall'] = recall
            
            wandb.log({'test_' + label_unmapping[c] + '_f1': f1})
            wandb.log({'test_' + label_unmapping[c] + '_precision': precision})
            wandb.log({'test_' + label_unmapping[c] + '_recall': recall})

        
        plot_confusion_matrix(conf_mx,fold,classnames, title='Test Data Confusion matrix - ViewNet', 
                                      cmap=plt.cm.Blues)
        
        average_types = ['macro', 'micro', 'weighted']
        average_metrics_to_log = ['precision', 'recall', 'f1score', 'support']
        average_dict = {'fold': fold}
        for av in average_types:
            results_tuple = precision_recall_fscore_support(run_y, run_pred, average=av)
            for m in range(len(average_metrics_to_log)):      
                #average_dict[ 'test_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                test_metrics['test_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
        test_metrics['test_acc_average'] = accuracy_score(run_y, run_pred)                  
        #average_dict[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)     
        #wandb.log(test_metrics)
        wandb.config.update(test_metrics)

        
        # saving results in df
        final_df['IDs_iter_' + str(fold)] = np.array(run_ids)
        final_df['labels_iter_' + str(fold)] = np.array(run_y)
        final_df['preds_iter_' + str(fold)] = np.array(run_pred)
        
        test_metrics_per_fold.append(test_metrics)
        
                

    # Calculate the performance metrics on the best model in each fold
    wandb.init(project=project_name, entity=wandb_username, name=local_username + '_ALL', group=random_str)
    config_dict['fold'] = -1
#     wandb.config.update(config_dict)
#     wandb.config.update(network_configs)
    final_df.to_csv (r'final_viewnet_data.csv', index = False, header=True)

    metrics_all = {}
    for fold in best_metrics_per_fold:
        for key in fold:
            if key not in metrics_all:
                metrics_all[key] = [fold[key]]
            else:
                metrics_all[key].append(fold[key]) 
                
    # add test data to metrics_all
    for fold in test_metrics_per_fold:
        for key in fold:
            if key not in metrics_all:
                metrics_all[key] = [fold[key]]
            else:
                metrics_all[key].append(fold[key]) 

    metrics_to_log = {}
    for m in metrics_all:
        try:
            
            metric_list = np.asarray(metrics_all[m])
    #         print(m)
    #         print(metric_list)
    #         print(type(metric_list))
            metrics_to_log[m + '_mean'] = metric_list.mean()    
            metrics_to_log[m + '_stdev'] = metric_list.std()
        except: 
            pass
    wandb.config.update(metrics_to_log)
  

In [ ]:
repetitions = 1

conv1_filters = 8
conv2_filters = 16
conv3_filters = 32
linear1_size = 512

dropout = 0.25
lr = 0.0005
wd = 0.001
amsgrad = False
for i in range(repetitions):
    config_string = f"{conv1_filters}_{conv2_filters}_{conv3_filters}_{linear1_size}_{dropout}_{lr}_{wd}_{amsgrad}"
    model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
    run_configs = {'lr': lr, 'wd': wd, 'amsgrad': amsgrad,'dropout': dropout, 
                  'conv1_filters': conv1_filters, 'conv2_filters': conv2_filters, 
                  'conv3_filters': conv3_filters, 'linear1_size': linear1_size }

    train5fold(run_configs, model_ft, lr, wd, amsgrad, i)

# model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
# print(model_ft)
# num_parameters = sum(p.numel() for p in model_ft.parameters())
# print(num_parameters)

conv_output:  512


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


	 conv1.weight
	 conv1.bias
	 conv2.weight
	 conv2.bias
	 conv3.weight
	 conv3.bias
	 linear1.weight
	 linear1.bias
	 linear2.weight
	 linear2.bias
Epoch 1/100
------------------------------------------------------
train loss:	1.5739 | train acc:	0.3803

              precision    recall  f1-score   support

           0       0.38      0.89      0.53      3724
           1       0.40      0.09      0.15      1813
           2       0.04      0.00      0.00      1218
           3       0.34      0.22      0.27      1895
           4       0.27      0.01      0.02      1359
           5       0.58      0.27      0.37      1072

    accuracy                           0.38     11081
   macro avg       0.33      0.25      0.22     11081
weighted avg       0.34      0.38      0.29     11081

val loss:	1.5754 | val acc:	0.4188

{'val_Other_f1': 0.5181228538725677, 'val_Other_precision': 0.4843081312410842, 'val_Other_recall': 0.55701394585726, 'val_Saggital_Right_f1': 0.3909774436090226, 'va

train loss:	1.2550 | train acc:	0.5128

              precision    recall  f1-score   support

           0       0.52      0.74      0.61      3724
           1       0.49      0.36      0.41      1813
           2       0.45      0.15      0.22      1218
           3       0.45      0.49      0.47      1895
           4       0.46      0.30      0.36      1359
           5       0.70      0.73      0.72      1072

    accuracy                           0.51     11081
   macro avg       0.51      0.46      0.46     11081
weighted avg       0.50      0.51      0.49     11081

val loss:	1.4925 | val acc:	0.4143

{'val_Other_f1': 0.3948973932334997, 'val_Other_precision': 0.6095890410958904, 'val_Other_recall': 0.29204265791632483, 'val_Saggital_Right_f1': 0.4739429695181907, 'val_Saggital_Right_precision': 0.39834710743801655, 'val_Saggital_Right_recall': 0.5849514563106796, 'val_Transverse_Right_f1': 0.3208191126279863, 'val_Transverse_Right_precision': 0.35471698113207545, 'val_Transv

val loss:	1.3875 | val acc:	0.4974

{'val_Other_f1': 0.503690036900369, 'val_Other_precision': 0.5753424657534246, 'val_Other_recall': 0.4479081214109926, 'val_Saggital_Right_f1': 0.5953002610966057, 'val_Saggital_Right_precision': 0.6440677966101694, 'val_Saggital_Right_recall': 0.5533980582524272, 'val_Transverse_Right_f1': 0.4166666666666667, 'val_Transverse_Right_precision': 0.5737704918032787, 'val_Transverse_Right_recall': 0.32710280373831774, 'val_Saggital_Left_f1': 0.46424870466321244, 'val_Saggital_Left_precision': 0.3888888888888889, 'val_Saggital_Left_recall': 0.5758354755784062, 'val_Transverse_Left_f1': 0.3643122676579926, 'val_Transverse_Left_precision': 0.29577464788732394, 'val_Transverse_Left_recall': 0.47419354838709676, 'val_Bladder_f1': 0.6654411764705883, 'val_Bladder_precision': 0.5691823899371069, 'val_Bladder_recall': 0.8008849557522124, 'val_precision_average_macro': 0.5078377801466988, 'val_recall_average_macro': 0.5298871605199088, 'val_f1score_average_macro'

...

wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


	 conv1.weight
	 conv1.bias
	 conv2.weight
	 conv2.bias
	 conv3.weight
	 conv3.bias
	 linear1.weight
	 linear1.bias
	 linear2.weight
	 linear2.bias
Epoch 1/100
------------------------------------------------------
train loss:	1.5944 | train acc:	0.3700

              precision    recall  f1-score   support

           0       0.37      0.93      0.52      3756
           1       0.37      0.05      0.09      1813
           2       0.13      0.00      0.00      1284
           3       0.32      0.13      0.18      1717
           4       0.19      0.00      0.01      1286
           5       0.56      0.22      0.32      1009

    accuracy                           0.37     10865
   macro avg       0.32      0.22      0.19     10865
weighted avg       0.33      0.37      0.26     10865

val loss:	1.4947 | val acc:	0.5057

{'val_Other_f1': 0.6027596223674655, 'val_Other_precision': 0.5296745373324825, 'val_Other_recall': 0.6992417860151643, 'val_Saggital_Right_f1': 0.3835319609967497, '

val loss:	1.3518 | val acc:	0.5512

{'val_Other_f1': 0.6463207867073584, 'val_Other_precision': 0.5408626560726447, 'val_Other_recall': 0.8028643639427128, 'val_Saggital_Right_f1': 0.5195071868583162, 'val_Saggital_Right_precision': 0.4501779359430605, 'val_Saggital_Right_recall': 0.6140776699029126, 'val_Transverse_Right_f1': 0.3412322274881517, 'val_Transverse_Right_precision': 0.4311377245508982, 'val_Transverse_Right_recall': 0.2823529411764706, 'val_Saggital_Left_f1': 0.35459183673469385, 'val_Saggital_Left_precision': 0.6405529953917051, 'val_Saggital_Left_recall': 0.24514991181657847, 'val_Transverse_Left_f1': 0.296, 'val_Transverse_Left_precision': 0.6324786324786325, 'val_Transverse_Left_recall': 0.19321148825065274, 'val_Bladder_f1': 0.7684021543985636, 'val_Bladder_precision': 0.7985074626865671, 'val_Bladder_recall': 0.740484429065744, 'val_precision_average_macro': 0.5822862345205847, 'val_recall_average_macro': 0.47969013402584526, 'val_f1score_average_macro': 0.487675698

train loss:	1.2119 | train acc:	0.5216

              precision    recall  f1-score   support

           0       0.51      0.72      0.60      3756
           1       0.56      0.42      0.48      1813
           2       0.48      0.28      0.35      1284
           3       0.45      0.44      0.44      1717
           4       0.45      0.28      0.34      1286
           5       0.73      0.73      0.73      1009

    accuracy                           0.52     10865
   macro avg       0.53      0.48      0.49     10865
weighted avg       0.52      0.52      0.51     10865

val loss:	1.2759 | val acc:	0.5858

{'val_Other_f1': 0.64953080375357, 'val_Other_precision': 0.629746835443038, 'val_Other_recall': 0.6705981465880371, 'val_Saggital_Right_f1': 0.4803493449781659, 'val_Saggital_Right_precision': 0.6, 'val_Saggital_Right_recall': 0.40048543689320387, 'val_Transverse_Right_f1': 0.42506811989100823, 'val_Transverse_Right_precision': 0.6964285714285714, 'val_Transverse_Right_recall':

train loss:	1.1427 | train acc:	0.5568

              precision    recall  f1-score   support

           0       0.54      0.74      0.62      3756
           1       0.61      0.46      0.53      1813
           2       0.56      0.34      0.42      1284
           3       0.49      0.46      0.47      1717
           4       0.48      0.35      0.40      1286
           5       0.76      0.77      0.76      1009

    accuracy                           0.56     10865
   macro avg       0.57      0.52      0.53     10865
weighted avg       0.56      0.56      0.55     10865

val loss:	1.2249 | val acc:	0.6104

{'val_Other_f1': 0.6640127388535032, 'val_Other_precision': 0.6294339622641509, 'val_Other_recall': 0.7026116259477675, 'val_Saggital_Right_f1': 0.5007451564828614, 'val_Saggital_Right_precision': 0.6486486486486487, 'val_Saggital_Right_recall': 0.4077669902912621, 'val_Transverse_Right_f1': 0.5211581291759465, 'val_Transverse_Right_precision': 0.6030927835051546, 'val_Transvers